<a href="https://colab.research.google.com/github/simon19891101/ML_experiments/blob/main/LLM_search_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers accelerate langchain sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## LLM for enhancing the search query

In [4]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=False,
    top_k=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [18]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [58]:
prompt='What are the best gifts for boys under 5?'

template='''<s>[INST] <<SYS>>
Only tell me the product names. The answer should only include ten names.
<</SYS>>

{prompt}[/INST]'''

prompt_template = PromptTemplate(template=template, input_variables=["prompt"])

llm = HuggingFacePipeline(pipeline=pipeline)

llm_chain = LLMChain(prompt=prompt_template, llm=llm)

print(llm_chain.run(prompt))


  Sure, here are ten popular product names for gifts for boys under 5:

1. LEGO
2. Hot Wheels
3. Play-Doh
4. Thomas the Tank Engine
5. Nerf
6. Paw Patrol
7. Disney Cars
8. Monster Trucks
9. Building Blocks
10. Crayola


## Build the Vector DB for products

In [24]:
import pandas as pd

In [26]:
#https://data.world/opensnippets/ebay-uk-products-dataset
product_names = pd.read_csv('products.csv', usecols=['name'])

In [61]:
product_names.head()

,name
0,2x Ultra-thin LCD Digital Display Vehicle Car ...
1,MERCEDES VANEO 414 1.6 Pollen / Cabin Filter 0...
2,Windscreen Kappa Transparent 58x45cm for April...
3,HP LaserJet Enterprise M552DN Colour 1200 X 12...
4,Honda RS 250 R 1991-1994 DP Brakes Street Sint...


In [28]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [29]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device': 'cpu'})

In [33]:
documents = product_names['name'].values.astype(str)
vectorstore = FAISS.from_texts(documents, embeddings)

## Similarity search

In [66]:
prompt='What are the best gifts for boys under 5?'
query = llm_chain.run(prompt)
print(query)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Sure, here are ten popular product names for gifts for boys under 5:

1. LEGO
2. Hot Wheels
3. Play-Doh
4. Thomas the Tank Engine
5. Nerf
6. Paw Patrol
7. Disney Cars
8. Monster Trucks
9. Building Blocks
10. Crayola


In [65]:
vectorstore.similarity_search_with_score(query, k=10)

[(Document(page_content='Count The Toys a Fun Picture Adding up Book for 2-5 Year Olds (pb) 1976832578', metadata={}),
  0.9526259),
 (Document(page_content='Mega Craft Jar Arts and Crafts Kit for Kids Over 1 500 Pieces Children Aged 3', metadata={}),
  1.0241055),
 (Document(page_content='Intex Underwater Fish Dive Rings Kids Swimming Pool Diving Toy Play Sticks 4 Set', metadata={}),
  1.1240711),
 (Document(page_content='Lovable Amigurumi Toys: 15 Doll Crochet Projects by Lilleliis by Mari-Liis Lille (Paperback, 2019)', metadata={}),
  1.1408081),
 (Document(page_content='Cutetitos Fruititos 39144 Surprise Stuffed Animals Cute Plush Surprise Toys 2', metadata={}),
  1.1448946),
 (Document(page_content='Crayola Shimmer Activity Pack 3 Books and 90 Stickers', metadata={}),
  1.159426),
 (Document(page_content='LEGO 76395 Harry Potter Hogwarts First Flying Lesson Age 7 264pcs', metadata={}),
  1.1774634),
 (Document(page_content='Disney Hotshots Princess Fashion Boutique - PC Cd-rom Gam